In [1]:
import sys

sys.path.insert(0, '/home/jianx/search-exposure/')
import torch
from annoy import AnnoyIndex
import forward_ranker.load_data as load_data
import forward_ranker.train as train
from forward_ranker.utils import print_message
from forward_ranker.test import get_ndcg_precision_rr
obj_reader = load_data.obj_reader
obj_writer = load_data.obj_writer
import pickle
import numpy as np
import random
import math

In [2]:
GROUND_TRUTH_PATH = "/datadrive/jianx/data/results/all_search_rankings_100_100_flat.csv"

In [3]:
def load_ground_truth(path=GROUND_TRUTH_PATH):
    all_results = {}
    with open(path, "r") as f:
        for line in f:
            qid = int(line.split(",")[0])
            pid = int(line.split(",")[1])
            rank = int(line.split(",")[2])
            if pid not in all_results.keys():
                all_results[pid] = {}
            all_results[pid][qid] = 101 - rank
    return all_results

In [4]:
ratings = load_ground_truth(GROUND_TRUTH_PATH)

In [5]:
baseline_reverse = obj_reader("/datadrive/ruohan/reverse_ranker/bug_fixed/forward_baseline_rank_test.pickle")
trained_reverse = obj_reader("/datadrive/ruohan/reverse_ranker/bug_fixed/pred_rank_test.pickle")
transformed_reverse = obj_reader("/datadrive/ruohan/reverse_ranker/transformation/pred_rank_test.pickle")

In [7]:
baseline_reverse_random = obj_reader("/datadrive/ruohan/reverse_ranker/append/layer_6_optim_fixed/forward_baseline_rank_test.pickle")
trained_reverse_random = obj_reader("/datadrive/ruohan/reverse_ranker/append/layer_6_optim_fixed/pred_rank_test.pickle")

In [8]:
baseline_reverse_residual = obj_reader("/datadrive/ruohan/reverse_ranker/residual/loss15_layer3/forward_baseline_rank_test.pickle")
trained_reverse_residual = obj_reader("/datadrive/ruohan/reverse_ranker/residual/loss15_layer3/pred_rank_test.pickle")

In [9]:
def get_reverse_ndcg_precision_rr(true_dict, test_dict, rank):
    sorted_result = list(test_dict.items())
    original_rank = rank
    rank = min(rank, len(sorted_result))
    cumulative_gain = 0
    num_positive = 0
    rr = float("NaN")
    for i in range(len(sorted_result)):
        pid = sorted_result[i][0]
        if pid in true_dict:
            rr = 1 / (i + 1)
            break
    for i in range(rank):
        pid = sorted_result[i][0]
        if pid in true_dict:
            num_positive += 1
    for i in range(rank):
        pid = sorted_result[i][0]
        relevance = 0
        if pid in true_dict:
            relevance = true_dict[pid]
        discounted_gain = relevance / math.log2(2 + i)
        cumulative_gain += discounted_gain
    sorted_ideal = sorted(true_dict.items(), key=lambda x: x[1], reverse=True)
    ideal_gain = 0
    for i in range(rank):
        relevance = 0
        if i < len(sorted_ideal):
            relevance = sorted_ideal[i][1]
        discounted_gain = relevance / math.log2(2 + i)
        ideal_gain += discounted_gain
    ndcg = 0
    if ideal_gain != 0:
         ndcg = cumulative_gain / ideal_gain
    return ndcg, num_positive / original_rank, rr

def calculate_metrics(rating_dict, result_dict, rank=10):
    pids = list(result_dict.keys())
    result_ndcg = []
    result_prec = []
    result_rr = []
    for pid in pids:
        if pid in rating_dict:
            ndcg, prec, rr = get_reverse_ndcg_precision_rr(rating_dict[pid], result_dict[pid], rank)
            result_ndcg.append(ndcg)
            result_prec.append(prec)
            result_rr.append(rr)
    avg_ndcg = np.nanmean(result_ndcg)
    avg_prec = np.nanmean(result_prec)
    avg_rr = np.nanmean(result_rr)
    print("NDCG@{}: {:.4f}".format(rank,avg_ndcg),"Precision@{}: {:.4f}".format(rank, avg_prec), "RR: {:.4f}".format(avg_rr))

In [10]:
print("Baseline forward embedding model")
calculate_metrics(ratings, baseline_reverse)
calculate_metrics(ratings, baseline_reverse, 100)
print("Append embedding model")
calculate_metrics(ratings, trained_reverse)
calculate_metrics(ratings, trained_reverse, 100)
print("Transformation model")
calculate_metrics(ratings, transformed_reverse)
calculate_metrics(ratings, transformed_reverse, 100)

Baseline forward embedding model
NDCG@10: 0.6232 Precision@10: 0.3208 RR: 0.7877
NDCG@100: 0.6932 Precision@100: 0.0597 RR: 0.7877
Append embedding model
NDCG@10: 0.7310 Precision@10: 0.3732 RR: 0.8474
NDCG@100: 0.7898 Precision@100: 0.0661 RR: 0.8474
Transformation model
NDCG@10: 0.2047 Precision@10: 0.1030 RR: 0.3943
NDCG@100: 0.2693 Precision@100: 0.0243 RR: 0.3943


In [11]:
print("Random baseline forward embedding model")
calculate_metrics(ratings, baseline_reverse_random)
calculate_metrics(ratings, baseline_reverse_random, 100)
print("Random append embedding model")
calculate_metrics(ratings, trained_reverse_random)
calculate_metrics(ratings, trained_reverse_random, 100)

Random baseline forward embedding model
NDCG@10: 0.6218 Precision@10: 0.3244 RR: 0.7892
NDCG@100: 0.6915 Precision@100: 0.0605 RR: 0.7892
Random append embedding model
NDCG@10: 0.8110 Precision@10: 0.4149 RR: 0.9073
NDCG@100: 0.8533 Precision@100: 0.0693 RR: 0.9073


In [12]:
print("Residual baseline forward embedding model")
calculate_metrics(ratings, baseline_reverse_residual)
calculate_metrics(ratings, baseline_reverse_residual, 100)
print("Residual embedding model")
calculate_metrics(ratings, trained_reverse_residual)
calculate_metrics(ratings, trained_reverse_residual, 100)

Residual baseline forward embedding model
NDCG@10: 0.6218 Precision@10: 0.3244 RR: 0.7892
NDCG@100: 0.6915 Precision@100: 0.0605 RR: 0.7892
Residual embedding model
NDCG@10: 0.8204 Precision@10: 0.4183 RR: 0.9096
NDCG@100: 0.8597 Precision@100: 0.0694 RR: 0.9096


In [ ]:
calculate_metrics(ratings, new_baseline_reverse)
calculate_metrics(ratings, new_baseline_reverse,100)

calculate_metrics(ratings, new_trained_reverse)
calculate_metrics(ratings, new_trained_reverse, 100)

In [48]:
import random
train_results = {1:[(1,2),(3,4)],2:[(5,6),(3,4)],3:[(7,8),(3,4)]}
train_set = dict(random.sample(list(train_results.items()), 2))
for key,value in train_set.items():
    for qid,rank in value:
        print(key,qid,rank)

2 5 6
2 3 4
1 1 2
1 3 4
